In [1]:
pip install mip

In [3]:
from mip import Model, xsum, maximize, BINARY
from more_itertools import locate

In [4]:
objects = []
weights = []
prices = []
volumes = []
W = 1

with open('opt2.txt') as txt:
  for line in txt:
    l = line.rstrip().split()
    objects.append(l[0])
    prices.append(float(l[1]))
    weights.append(float(l[2]))
    volumes.append(float(l[3]))

depends = {}
with open('opt2_dep.txt') as txt:
  for line in txt:
    l = line.rstrip().split()
    depends[l[1]] = l[0]

over = {}
with open('opt2_over.txt') as txt:
  for line in txt:
    l = line.rstrip().split()
    for i in range(len(l)-1):
      over[l[i]] = float(l[-1])

**base code**

In [5]:
W = 0.5
m = Model("knapsack")
x = [m.add_var(var_type=BINARY) for i in range(len(objects))]
m.objective = maximize(xsum(prices[i] * x[i] for i in range(len(objects))))
m += xsum(weights[i] * x[i] for i in range(len(objects))) <= W

m.optimize()
selected = [i for i in range(len(objects)) if x[i].x >= 0.99]
sels = []

for ind in selected:
  sels.append(objects[ind])

print(", ".join(sels))

Laptop


In a regular case the most expensive thing possible to have is a laptop

**with dependencies**

In [6]:
W = 0.5
m = Model('knapsack')
x = [m.add_var(var_type=BINARY) for i in range(len(objects))]
m.objective = maximize(xsum(prices[i] * x[i] for i in range(len(objects))))
m += xsum(weights[i] * x[i] for i in range(len(objects))) <= W

for k, v in depends.items():
    m += xsum(x[i] for i in (list(locate(objects, lambda j: j == v)))) >= x[objects.index(k)]

m.optimize()
selected = [i for i in range(len(objects)) if x[i].x >= 0.99]

sels = []
for ind in selected:
  sels.append(objects[ind])

print(", ".join(sels))


Flashcard, Cryptokey, Battery


with added dependencies, laptop canot be taken just by itself, so the taken objects are changed

**with oversize items**

In [7]:
#over

In [13]:
W = 0.58
m = Model('knapsack')
x = [m.add_var(var_type=BINARY) for i in range(len(objects))]
m.objective = maximize(xsum(prices[i] * x[i] for i in range(len(objects))))
m += xsum(weights[i] * x[i] for i in range(len(objects))) <= W


In [14]:
y = {}
for key in over.keys():
  y[key] = [m.add_var(var_type=BINARY) for i in range(len(objects))]

m.objective = maximize(xsum(prices[i] * x[i] for i in range(len(objects))))
m += xsum(weights[i] * x[i] for i in range(len(objects))) <= W

for key, value in over.items():
  for i in range(len(objects)):
    m += x[i] + x[objects.index(key)] - y[key][i] <= 1
    if i != objects.index(key):
      m += volumes[i] * y[key][i] <= value

m.optimize()
selected = [i for i in range(len(objects)) if x[i].x >= 0.99]

sels = []
for ind in selected:
  sels.append(objects[ind])

print(", ".join(sels))

Laptop, Flashcard, Cryptokey


Laptop (10) + Flashcard (5) + Cryptokey (1) are equal to 0.52. Battery or charger can fit by weight (0.05 and 0.01 are less than the place that is rest = 0.06), but the laptop hav the volume that persist objects with volume higher than 0.05 to be taken. 